##  AutoAI 데이터 생성
데이터 내용에도 함께 로컬라이징 적용

2020-09-23 학습 데이터 건수를 줄임(Lite Plan 적용)

In [1]:
# 공통 처리

# 라이브러리 임포트
import pandas as pd
# 데이터프레임 출력용 함수
from IPython.display import display

In [6]:
# 데이터 집합을 내려받아 압축 해제
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip -O bank.zip | tail -n 1
!unzip -o bank.zip | tail -n 1

# 역주: 위 명령에서 오류가 날 경우 URL의 파일을 직접 내려받아 notebooks 디렉토리에
# 압축을 해제하면 정상 진행할 수 있습니다.

# bank-full.csv 파일을 데이터프레임으로 읽어 들이기
df_all = pd.read_csv('bank-full.csv', sep=';')

# 우리말 필드명을 정의
columns = [
    '연령', '직업', '혼인_여부', '학력', '채무불이행', '평균잔고', 
    '주택대출', '신용대출', '연락수단', '마지막통화일', 
    '마지막통화월', '마지막통화시간', '통화횟수_캠페인중', 
    '마지막영업후_경과일수', '통화횟수_캠페인전', '지난영업_결과', 
    '이번영업_결과' 
]
df_all.columns = columns

'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
unzip:  cannot find either bank.zip or bank.zip.zip.
'tail'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [7]:
display(df_all.head())

,연령,직업,혼인_여부,학력,채무불이행,평균잔고,주택대출,신용대출,연락수단,마지막통화일,마지막통화월,마지막통화시간,통화횟수_캠페인중,마지막영업후_경과일수,통화횟수_캠페인전,지난영업_결과,이번영업_결과
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [8]:
yn_dict = {
    'yes': '예',
    'no': '아니오'
}
yn_dict2 = {
    'yes': '성공',
    'no': '실패'
}    

In [9]:
df_kr = df_all.copy()
df_kr['채무불이행'] = df_kr['채무불이행'].map(yn_dict)
df_kr['주택대출'] = df_kr['주택대출'].map(yn_dict)
df_kr['신용대출'] = df_kr['신용대출'].map(yn_dict)
df_kr['이번영업_결과'] = df_kr['이번영업_결과'].map(yn_dict2)

In [10]:
display(df_kr.head())

,연령,직업,혼인_여부,학력,채무불이행,평균잔고,주택대출,신용대출,연락수단,마지막통화일,마지막통화월,마지막통화시간,통화횟수_캠페인중,마지막영업후_경과일수,통화횟수_캠페인전,지난영업_결과,이번영업_결과
0,58,management,married,tertiary,아니오,2143,예,아니오,unknown,5,may,261,1,-1,0,unknown,실패
1,44,technician,single,secondary,아니오,29,예,아니오,unknown,5,may,151,1,-1,0,unknown,실패
2,33,entrepreneur,married,secondary,아니오,2,예,예,unknown,5,may,76,1,-1,0,unknown,실패
3,47,blue-collar,married,unknown,아니오,1506,예,아니오,unknown,5,may,92,1,-1,0,unknown,실패
4,33,unknown,single,unknown,아니오,1,아니오,아니오,unknown,5,may,198,1,-1,0,unknown,실패


In [12]:
random_seed = 123
# 학습 데이터와 검증 데이터를 분할
# 학습 데이터 10%, 검증 데이터 90%의 비율로 분할
test_size = 0.90

y = df_kr['이번영업_결과'].values

from sklearn.model_selection import train_test_split
df_kr_train, df_kr_test = train_test_split(
  df_kr, test_size=test_size, random_state=random_seed,
  stratify=y)

In [13]:
print(df_kr_train.shape)
print(df_kr_test.shape)

(4521, 17)
(40690, 17)


### CSV 파일로 저장하기

In [14]:
fn_train = 'bank-train-kr-autoai.csv'
df_kr_train.to_csv(fn_train, index=False)

In [15]:
fn_test = 'bank-test-kr-autoai.csv'
df_kr_test.to_csv(fn_test, index=False)

### Google Colab에서 내려받기

In [16]:
try:
    from google.colab import files
    files.download(fn_train) 
    files.download(fn_test)      
except:
    print('not google colab')

not google colab


### 여기서부터는 JSON 데이터 생성용 코드

In [18]:
df_kr_yes = df_kr_test[df_kr_test['이번영업_결과']=='성공']
display(df_kr_yes.head())

,연령,직업,혼인_여부,학력,채무불이행,평균잔고,주택대출,신용대출,연락수단,마지막통화일,마지막통화월,마지막통화시간,통화횟수_캠페인중,마지막영업후_경과일수,통화횟수_캠페인전,지난영업_결과,이번영업_결과
40231,25,services,single,tertiary,아니오,12,아니오,아니오,cellular,12,jun,126,6,-1,0,unknown,성공
43673,34,services,married,secondary,아니오,1076,아니오,아니오,cellular,12,may,152,1,182,6,success,성공
42398,68,retired,married,tertiary,아니오,2820,아니오,아니오,cellular,3,dec,154,1,97,2,failure,성공
43387,27,student,single,secondary,아니오,6036,아니오,아니오,cellular,31,mar,175,1,181,1,failure,성공
42163,54,technician,married,secondary,아니오,0,아니오,아니오,cellular,6,nov,464,5,192,8,other,성공


In [20]:
df_kr_no = df_kr_test[df_kr_test['이번영업_결과']=='실패']
display(df_kr_no.head())

,연령,직업,혼인_여부,학력,채무불이행,평균잔고,주택대출,신용대출,연락수단,마지막통화일,마지막통화월,마지막통화시간,통화횟수_캠페인중,마지막영업후_경과일수,통화횟수_캠페인전,지난영업_결과,이번영업_결과
30887,45,blue-collar,divorced,primary,아니오,1512,예,아니오,cellular,9,feb,13,11,-1,0,unknown,실패
16821,35,entrepreneur,single,tertiary,아니오,-880,아니오,예,cellular,24,jul,77,3,-1,0,unknown,실패
3011,36,services,married,secondary,아니오,338,아니오,아니오,unknown,14,may,90,1,-1,0,unknown,실패
17639,32,services,married,secondary,아니오,2578,아니오,아니오,cellular,29,jul,86,3,-1,0,unknown,실패
28856,39,management,married,secondary,아니오,1585,예,아니오,cellular,30,jan,82,2,-1,0,unknown,실패


In [21]:
df_kr_yes20 = df_kr_yes.drop('이번영업_결과', axis=1).head(20)
df_kr_no20 = df_kr_no.drop('이번영업_결과', axis=1).head(20)

In [22]:
fields = df_kr_yes20.columns.tolist()
print(fields)

['연령', '직업', '혼인_여부', '학력', '채무불이행', '평균잔고', '주택대출', '신용대출', '연락수단', '마지막통화일', '마지막통화월', '마지막통화시간', '통화횟수_캠페인중', '마지막영업후_경과일수', '통화횟수_캠페인전', '지난영업_결과']


In [23]:
values_yes = df_kr_yes20.values.tolist()
print(values_yes)

[[25, 'services', 'single', 'tertiary', '아니오', 12, '아니오', '아니오', 'cellular', 12, 'jun', 126, 6, -1, 0, 'unknown'], [34, 'services', 'married', 'secondary', '아니오', 1076, '아니오', '아니오', 'cellular', 12, 'may', 152, 1, 182, 6, 'success'], [68, 'retired', 'married', 'tertiary', '아니오', 2820, '아니오', '아니오', 'cellular', 3, 'dec', 154, 1, 97, 2, 'failure'], [27, 'student', 'single', 'secondary', '아니오', 6036, '아니오', '아니오', 'cellular', 31, 'mar', 175, 1, 181, 1, 'failure'], [54, 'technician', 'married', 'secondary', '아니오', 0, '아니오', '아니오', 'cellular', 6, 'nov', 464, 5, 192, 8, 'other'], [33, 'management', 'married', 'tertiary', '아니오', 273, '예', '아니오', 'unknown', 16, 'may', 803, 1, -1, 0, 'unknown'], [73, 'retired', 'married', 'tertiary', '아니오', 459, '아니오', '아니오', 'cellular', 17, 'feb', 281, 1, -1, 0, 'unknown'], [34, 'management', 'married', 'tertiary', '아니오', 1778, '아니오', '아니오', 'cellular', 4, 'nov', 358, 1, 162, 2, 'failure'], [56, 'retired', 'divorced', 'tertiary', '아니오', 158, '아니오', '예', 'cellu

In [24]:
values_no = df_kr_no20.values.tolist()
print(values_no)

[[45, 'blue-collar', 'divorced', 'primary', '아니오', 1512, '예', '아니오', 'cellular', 9, 'feb', 13, 11, -1, 0, 'unknown'], [35, 'entrepreneur', 'single', 'tertiary', '아니오', -880, '아니오', '예', 'cellular', 24, 'jul', 77, 3, -1, 0, 'unknown'], [36, 'services', 'married', 'secondary', '아니오', 338, '아니오', '아니오', 'unknown', 14, 'may', 90, 1, -1, 0, 'unknown'], [32, 'services', 'married', 'secondary', '아니오', 2578, '아니오', '아니오', 'cellular', 29, 'jul', 86, 3, -1, 0, 'unknown'], [39, 'management', 'married', 'secondary', '아니오', 1585, '예', '아니오', 'cellular', 30, 'jan', 82, 2, -1, 0, 'unknown'], [57, 'blue-collar', 'married', 'secondary', '아니오', 0, '예', '아니오', 'cellular', 14, 'may', 125, 5, -1, 0, 'unknown'], [33, 'management', 'married', 'tertiary', '아니오', 452, '예', '아니오', 'cellular', 29, 'aug', 168, 2, -1, 0, 'unknown'], [30, 'self-employed', 'married', 'secondary', '아니오', 8563, '예', '아니오', 'cellular', 12, 'may', 109, 1, 351, 4, 'other'], [42, 'management', 'single', 'secondary', '아니오', 9720, '아니오', '아

In [26]:
import json

In [27]:
scoring_payload_yes = {"input_data": [{
   "fields": fields, 
    "values": values_yes
}]}

print(json.dumps(scoring_payload_yes,  ensure_ascii=False))

{"input_data": [{"fields": ["연령", "직업", "혼인_여부", "학력", "채무불이행", "평균잔고", "주택대출", "신용대출", "연락수단", "마지막통화일", "마지막통화월", "마지막통화시간", "통화횟수_캠페인중", "마지막영업후_경과일수", "통화횟수_캠페인전", "지난영업_결과"], "values": [[25, "services", "single", "tertiary", "아니오", 12, "아니오", "아니오", "cellular", 12, "jun", 126, 6, -1, 0, "unknown"], [34, "services", "married", "secondary", "아니오", 1076, "아니오", "아니오", "cellular", 12, "may", 152, 1, 182, 6, "success"], [68, "retired", "married", "tertiary", "아니오", 2820, "아니오", "아니오", "cellular", 3, "dec", 154, 1, 97, 2, "failure"], [27, "student", "single", "secondary", "아니오", 6036, "아니오", "아니오", "cellular", 31, "mar", 175, 1, 181, 1, "failure"], [54, "technician", "married", "secondary", "아니오", 0, "아니오", "아니오", "cellular", 6, "nov", 464, 5, 192, 8, "other"], [33, "management", "married", "tertiary", "아니오", 273, "예", "아니오", "unknown", 16, "may", 803, 1, -1, 0, "unknown"], [73, "retired", "married", "tertiary", "아니오", 459, "아니오", "아니오", "cellular", 17, "feb", 281, 1, -1, 0, "unknown"],

In [28]:
scoring_payload_no = {"input_data": [{
   "fields": fields, 
    "values": values_no
}]}

print(json.dumps(scoring_payload_no,  ensure_ascii=False))

{"input_data": [{"fields": ["연령", "직업", "혼인_여부", "학력", "채무불이행", "평균잔고", "주택대출", "신용대출", "연락수단", "마지막통화일", "마지막통화월", "마지막통화시간", "통화횟수_캠페인중", "마지막영업후_경과일수", "통화횟수_캠페인전", "지난영업_결과"], "values": [[45, "blue-collar", "divorced", "primary", "아니오", 1512, "예", "아니오", "cellular", 9, "feb", 13, 11, -1, 0, "unknown"], [35, "entrepreneur", "single", "tertiary", "아니오", -880, "아니오", "예", "cellular", 24, "jul", 77, 3, -1, 0, "unknown"], [36, "services", "married", "secondary", "아니오", 338, "아니오", "아니오", "unknown", 14, "may", 90, 1, -1, 0, "unknown"], [32, "services", "married", "secondary", "아니오", 2578, "아니오", "아니오", "cellular", 29, "jul", 86, 3, -1, 0, "unknown"], [39, "management", "married", "secondary", "아니오", 1585, "예", "아니오", "cellular", 30, "jan", 82, 2, -1, 0, "unknown"], [57, "blue-collar", "married", "secondary", "아니오", 0, "예", "아니오", "cellular", 14, "may", 125, 5, -1, 0, "unknown"], [33, "management", "married", "tertiary", "아니오", 452, "예", "아니오", "cellular", 29, "aug", 168, 2, -1, 0, "unkno

In [29]:
print(df_kr_test.shape)

(40690, 17)
